In [16]:
from backend.resident_model import cal_future_affordability_index
from backend.government_model import cal_future_tax_revenue_index
from backend.developer_model import cal_future_profit_index
from backend.input_data import refresh_input

input_json = {"bcr": 0.9, "tier": 3, "residential": 0.0, "office": 0.5, "amenity": 0.2, "civic": 0.3}
refresh_input(input_json)
affordability_index = cal_future_affordability_index()
tax_revenue_index = cal_future_tax_revenue_index()
profit_index = cal_future_profit_index()
print(affordability_index,tax_revenue_index,profit_index)

/Users/kai/Library/CloudStorage/GoogleDrive-kaihu@media.mit.edu/My Drive/Generative-Metrics-main/output/input.json
office_space: 2753.3700000000003 amenity_space: 1835.5800000000004 civic_space: 2753.3700000000003 resident_space: 1835.5800000000004 open_space: 27533.7 green_space: 13766.85


TypeError: refresh_input() takes 0 positional arguments but 1 was given

In [ ]:
import numpy as np
import math
def cal_surface_to_volume_ratio(state):
    matrix = (state!=0)
    volume = np.sum(matrix)
    surface_area = 0
    # 在三个维度上都填充一个0边界
    padded_matrix = np.pad(matrix, pad_width=1, mode='constant', constant_values=0)
    # 对于每一个方向，检查邻居是否存在
    directions = [(1, 0, 0), (-1, 0, 0), (0, 1, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1)]
    for dx, dy, dz in directions:
        # 使用numpy的roll函数来移动数组，使得对应的邻居可以对齐
        neighbor = np.roll(padded_matrix, shift=(dx, dy, dz), axis=(0, 1, 2))
        surface_area += np.sum(padded_matrix * (padded_matrix != neighbor))
    surface_to_volume_ratio = surface_area/volume
    return surface_to_volume_ratio

def cal_surface_to_volume_index(state):
    surface_to_volume_ratio = cal_surface_to_volume_ratio(state)
    target_ratio = 0.3
    sigma = 1
    score = math.exp(-((surface_to_volume_ratio  - target_ratio) ** 2) / (2 * sigma ** 2))
    surface_to_volume_index = 100 * score
    return surface_to_volume_index
    

In [ ]:
import numpy as np
def cal_sky_view(state):
    matrix = (state!=0)
    sky_view = np.zeros((10, 10), dtype=int)
    for x in range(10):
        for y in range(10):
            if matrix[x, y, 0] == 0:
                directions = [
                    (1, 0, 2),   # x 30
                    (-1, 0, 2),  # x -30
                    (0, 1, 2),   # y 30
                    (0, -1, 2),  # y -30
                    (0, 0, 1)    # z vertical
                ]
                for dx, dy, dz in directions:
                    x_current, y_current, z_current = x, y, 0
                    while 0 <= x_current < 10 and 0 <= y_current < 10 and 0 <= z_current < 10:
                        x_current += dx
                        y_current += dy
                        z_current += dz
                        if 0 <= x_current < 10 and 0 <= y_current < 10 and 0 <= z_current < 10:
                            if matrix[x_current, y_current, z_current] == 0:
                                sky_view[x, y] += 1
                            else:
                                break

    return sky_view
def cal_sky_view_index(state):
    sky_view = cal_sky_view(state)
    sky_view_index = np.sum(sky_view/25)
    return sky_view_index

[[ 0 17 18 20 21 21 20 18 17  0]
 [17 19 20 21 22 22 21 20 19 17]
 [18 20 21 22 23 23 22 21 20 18]
 [20 21 22 23 24 24 23 22 21 20]
 [21 22 23 24 25 25 24 23 22 21]
 [21 22 23 24 25  0  0  0  0  0]
 [20 21 22 23 24  0  0  0  0  0]
 [19 20 21 22 23  0  0  0  0  0]
 [18 19 20 21 22  0  0  0  0  0]
 [17 18 19 20 21  0  0  0  0  0]]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

def compute_score(value):
    mu = 0.3
    sigma = 1
    score = math.exp(-((value - mu) ** 2) / (2 * sigma ** 2))
    normalized_score = 100 * score
    return normalized_score


In [ ]:
def get_normalized_score(value,target,sigma=1):
    score = np.exp(-((value - target) ** 2) / (2 * sigma ** 2))
    normalized_score = score
    return normalized_score

In [ ]:
from PIL import Image
ref_img = Image.open('ref.png').resize((10,10)).convert('L')
ref_state = np.array(ref_img)/255.

state = np.count_nonzero(np.random.randint(0,10,(10,10,10)),axis=-1)/10

normalized_score = get_normalized_score(state,ref_state,sigma=0.1)
print(np.sum(normalized_score))

8.905195524210919


In [ ]:
def get_score(input_json):
    # api_address = "http://18.27.114.99:5001/"
    api_address = "http://127.0.0.1:5001/"
    #api
    requests.post(api_address+"api/save_data/input",json=input_json)
    compute = requests.post(api_address+"/api/compute")
    output = requests.get(api_address+"/api/get_data/index_score.json")
    score = output.json()
    df = pd.DataFrame(score)
    r_score = df['score'][df['stakeholder']=='Residents'].item()
    g_score = df['score'][df['stakeholder']=='Government'].item()
    d_score = df['score'][df['stakeholder']=='Developer'].item()
    return r_score,g_score,d_score

In [16]:
import requests
input_json = {"bcr": 0.8, "tier": 3, "residential": 0.1, "office": 0.4, "amenity": 0.2, "civic": 0.3}
server_url = "http://127.0.0.1:5000"
requests.post(server_url+'/api/receive_values',json=input_json)
requests.post(server_url+"/api/save_data/input",json=input_json)
requests.get(server_url+"/api/compute")
output = requests.get(server_url+"/api/get_data/index_score.json")
requests.get(server_url+"/api/refresh")
print(output.json())

[{'baseline': 77.32, 'indicator': 'Affordability', 'score': 83.83, 'stakeholder': 'Residents'}, {'baseline': 0.0, 'indicator': 'Tax revenue', 'score': 17.8673140567, 'stakeholder': 'Government'}, {'baseline': 0.0, 'indicator': 'Profit', 'score': 87.86, 'stakeholder': 'Developer'}, {'baseline': 0.55, 'indicator': 'Access to service', 'score': 0.1333703383, 'stakeholder': 'Industry Group'}, {'baseline': 0.55, 'indicator': 'Opportunity', 'score': 0.1333703383, 'stakeholder': 'Industry Group'}, {'baseline': 1.0, 'indicator': 'Safety & security', 'score': 0.7205648101, 'stakeholder': 'Industry Group'}, {'baseline': 0.0, 'indicator': 'Innovation', 'score': 0.7205648101, 'stakeholder': 'Industry Group'}, {'baseline': 0.3, 'indicator': 'Finance', 'score': 0.3539072762, 'stakeholder': 'Local Business Owners'}, {'baseline': 1.0, 'indicator': 'Safety & security', 'score': 0.8454806264, 'stakeholder': 'Local Business Owners'}, {'baseline': 0.55, 'indicator': 'Access to service', 'score': 0.5697712